# Model Registry Demo

## Setup Notebook and Import Path

In [ ]:
# Scale cell width with the browser window to accommodate .show() commands for wider tables.
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import sys
import os

# Simplify reading from the local repository
cwd=os.getcwd()
REPO_PREFIX="snowflake/ml"
LOCAL_REPO_PATH=cwd[:cwd.find(REPO_PREFIX)].rstrip('/')

if LOCAL_REPO_PATH not in sys.path:
    print(f"Adding {LOCAL_REPO_PATH} to system path")
    sys.path.append(LOCAL_REPO_PATH)

## Train A Small Model

The cell below trains a small model for demonstration purposes. The nature of the model does not matter, it is purely used to demonstrate the usage of the Model Registry.

In [ ]:
from sklearn import svm, linear_model
from sklearn.datasets import load_digits

digits = load_digits()
target_digit = 6
num_training_examples = 10
svc_gamma = 0.001
svc_C = 10.

clf = svm.SVC(gamma=svc_gamma, C=svc_C, probability=True)


def one_vs_all(dataset, digit):
    return [x == digit for x in dataset]

# Train a classifier using num_training_examples and use the last 100 examples for test.
train_features = digits.data[:num_training_examples]
train_labels = one_vs_all(digits.target[:num_training_examples], target_digit)
clf.fit(train_features, train_labels)

test_features = digits.data[-100:]
test_labels = one_vs_all(digits.target[-100:], target_digit)
prediction = clf.predict(test_features)


## Start Snowpark Session

To avoid exposing credentials in Github, we use a small utility `SnowflakeLoginOptions`. It allows you to score your default credentials in `~/.snowsql/config` in the following format:
```
[connections]
accountname = <string>   # Account identifier to connect to Snowflake.
username = <string>      # User name in the account. Optional.
password = <string>      # User password. Optional.
dbname = <string>        # Default database. Optional.
schemaname = <string>    # Default schema. Optional.
warehousename = <string> # Default warehouse. Optional.
#rolename = <string>      # Default role. Optional.
#authenticator = <string> # Authenticator: 'snowflake', 'externalbrowser', etc
```
Please follow [this](https://docs.snowflake.com/en/user-guide/snowsql-start.html#configuring-default-connection-settings) for more details.

In [ ]:
from snowflake.ml.utils.connection_params import SnowflakeLoginOptions
from snowflake.snowpark import Session, Column, functions

session = Session.builder.configs(SnowflakeLoginOptions()).create()

## Open/Create Model Registry

A model registry needs to be created before it can be used. The creation will create a new database in the current account so the active role needs to have permissions to create a database. After the first creation, the model registry can be opened without the need to create it again.

In [ ]:
import importlib
from snowflake.ml.registry import model_registry
# Force re-loading model_registry in case we updated the package during the runtime of this notebook.
importlib.reload(model_registry)

registry_name = "model_registry_zzhu"

In [ ]:
# Create a new model registry. This will be a no-op if the registry already exists.
create_result = model_registry.create_model_registry(session=session, database_name=registry_name)

In [ ]:
registry = model_registry.ModelRegistry(session=session, database_name=registry_name)

There are two functionally equivalent APIs to interact with the model registry.

* A _relational API_ where all operations are performed as methods of the `registry` object and 
* a _object API_ where operations on a specific model are performend as methods of a `ModelReference` object.

The usage examples below will add some color to the two APIs and how they behave.

## Register a new Model

Registering a new model is always performed through the relational API. 

The call to `log_model` executes a few steps:
1. The given model object is serialized and uploaded to a stage.
1. An entry in the Model Registry is created for the model, referencing the model stage location.
1. Additional metadata is updated for the model as provided in the call.

For the serialization to work, the model object needs to be serializable in python.

In [ ]:
# A name and model tags can be added to the model at registration time.
model_name = "my_model"
model_version = "108.2.4"

In [ ]:
model = registry.log_model(model_name=model_name, model_version=model_version, model=clf, tags={
    "stage": "testing", "classifier_type": "svm.SVC", "svc_gamma": svc_gamma, "svc_C": svc_C}, sample_input_data=train_features, options={"embed_local_ml_library": True})

In [ ]:
# The object API can be used to reference a model after creation.
model = model_registry.ModelReference(registry=registry, model_name=model_name, model_version=model_version)
print("Registered new model id:", model_id)

## Add Metrics

Metrics are a type of metadata annotation that can be associated with models stored in the Model Registry. Metrics often take the form of scalars but we also support more complex objects such as arrays or dictionaries to represent metrics. In the exmamples below, we add scalars, dictionaries, and a 2-dimensional numpy array as metrics.

In [ ]:
from sklearn import metrics

test_accuracy = metrics.accuracy_score(test_labels, prediction)
print("Model test accuracy:", test_accuracy)

# Simple scalar metrics.

# Relational API
registry.set_metric(model_name=model_name, model_version=model_version, metric_name="test_accuracy", metric_value=test_accuracy)

# Object API
model.set_metric(metric_name="num_training_examples", metric_value=num_training_examples)

# Hierarchical metric.
registry.set_metric(model_name=model_name, model_version=model_version, metric_name="dataset_test", metric_value={"accuracy": test_accuracy})

# Multivalent metric:
test_confusion_matrix = metrics.confusion_matrix(test_labels, prediction)
print("Confusion matrix:", test_confusion_matrix)

registry.set_metric(model_name=model_name, model_version=model_version, metric_name="confusion_matrix", metric_value=test_confusion_matrix)

In [ ]:
# Relational API
registry.get_metrics(model_name=model_name, model_version=model_version)

In [ ]:
# Object API
model.get_metrics()

## List Model in Registry

Listing models in the registry returns a SnowPark DataFrame. That allows the caller to select and filter the models as needed. In the example below, we list the name, tags, and metrics for the model we just added.

In [ ]:
model_list = registry.list_models()

model_list.filter(model_list["VERSION"] == model_version).select("NAME","VERSION","TAGS","METRICS").show()

## Metadata: Tags and Descriptions

Similar to how we changed metrics in the example above, we can also edit tags and descriptions of models both with the relational API and with the object API.

### Relational API

In [ ]:
print("Old tags:", registry.get_tags(model_name=model_name, model_version=model_version))

registry.set_tag(model_name=model_name, model_version=model_version, tag_name="minor_version", tag_value="23")
print("Added tag:", registry.get_tags(model_name=model_name, model_version=model_version,))

registry.remove_tag(model_name=model_name, model_version=model_version, tag_name="minor_version")
print("Removed tag", registry.get_tags(model_name=model_name, model_version=model_version,))

registry.set_tag(model_name, model_version,"stage","production")
print("Updated tag:", registry.get_tags(model_name=model_name, model_version=model_version,))

registry.set_model_description(description="My model is better than talkgpt-5!", model_name=model_name, model_version=model_version,)
print("Added description:", registry.get_model_description(model_name=model_name, model_version=model_version,))

### Object API

In [ ]:
print("Old tags:", model.get_tags())

model.set_tag("minor_version", "23")
print("Added tag:", model.get_tags())

model.remove_tag("minor_version")
print("Removed tag", model.get_tags())

model.set_tag("stage", "production")
print("Updated tag:", model.get_tags())

model.set_model_description(description="My model is better than speakgpt-6!")
print("New description:", model.get_model_description())

## List recent Models in Registry

Listing the models in the Model Registry returns a dataframe that allows us to conveniently manipulate the model list. In the example below, we show all models in the Model Registry sorted by recency.

In [ ]:
model_list.select("ID","NAME","VERSION","CREATION_TIME","TAGS").order_by("CREATION_TIME", ascending=False).show(3)

## List all versions of a Model ordered by test set accuracy

With a similar logic, we can also list all versions of a model with a given name sorted by a metric, in this case model accuracy.

In [ ]:
model_list.select("ID","NAME","VERSION","TAGS","METRICS").filter(
    Column("NAME") == model_name).order_by(Column("METRICS")["test_accuracy"], ascending=False 
).show(3)                                                                                                             

## Model Deployment
Registry can be used to create deployment, which can be used for prediction. Deployment exists in the form of UDF. It could be either permanent or temporary.


### Permanent deployment

In [ ]:
# Create a permanent deployment
model.deploy(deployment_name="PERM_DEPLOY_1_0", target_method="predict", permanent=True, options={"relax_version": True})

In [ ]:
# Create a permanent deployment with overridden UDF stage path
stage_path = f'"{registry_name}".PUBLIC._SYSTEM_REGISTRY_DEPLOYMENTS_VIEW_TEST'
session.sql(f"CREATE STAGE IF NOT EXISTS {stage_path}").collect()
model.deploy(deployment_name="PERM_DEPLOY_1_1", target_method="predict", permanent=True, options={"permanent_udf_stage_location":'@'+stage_path, "relax_version": True})

In [ ]:
model.list_deployments().select("MODEL_NAME", "MODEL_VERSION", "DEPLOYMENT_NAME").show()

In [ ]:
# Create another permanent deployment
model.deploy(deployment_name="PERM_DEPLOY_1_2", target_method="predict", permanent=True, options={"relax_version": True})

In [ ]:
model.predict("PERM_DEPLOY_1_0", test_features)

In [ ]:
model.predict("PERM_DEPLOY_1_1", test_features)

In [ ]:
model.predict("PERM_DEPLOY_1_2", test_features)

In [ ]:
model.delete_deployment(deployment_name="PERM_DEPLOY_1_2")

In [ ]:
model.list_deployments().select("MODEL_NAME", "MODEL_VERSION", "DEPLOYMENT_NAME").show()

### Temporary deployments


The key distinction between permanent and temporary deployments lies in their lifespan. Temporary deployments are session-scoped and get removed when the session ends. As a result, the methods `delete_deployment()` and `list_deployments()` currently do not support temporary deployments.

In [ ]:
# Create a temporary deployment
model.deploy(deployment_name="TEMP_DEPLOY_1_0", target_method="predict", permanent=False, options={"relax_version": True})

In [ ]:
# Create another temporary deployment
model.deploy(deployment_name="TEMP_DEPLOY_1_1", target_method="predict", permanent=False, options={"relax_version": True})

In [ ]:
model.predict("TEMP_DEPLOY_1_0", test_features)

In [ ]:
model.predict("TEMP_DEPLOY_1_1", test_features)

## Examine Model History

In addition to the current state of the model metadata, we also give access to the history of all changes to the model metadata. This includes the registration event itself but also changes to any metadata of the model, when they happened and who initiated them.

### Relational API

In [ ]:
registry.get_model_history(model_name=model_name, model_version=model_version).select("EVENT_TIMESTAMP", "ROLE", "ATTRIBUTE_NAME","OPERATION", "VALUE[ATTRIBUTE_NAME]").sort("EVENT_TIMESTAMP", ascending=False).show()

### Object API

In [ ]:
model.get_model_history().select("EVENT_TIMESTAMP", "ROLE", "ATTRIBUTE_NAME","OPERATION", "VALUE[ATTRIBUTE_NAME]").sort("EVENT_TIMESTAMP", ascending=False).show()